# Toronto Police Service API
- API documentation: https://developers.arcgis.com/rest/services-reference/enterprise/query-feature-service-layer-.htm 
  - While there is a limit to the number of features included in the feature set response, there is no limit to the number of object IDs returned in the ID array response. Clients can exploit this to get all the query conforming object IDs by specifying returnIdsOnly=true and subsequently requesting feature sets for subsets of object IDs.
  - **Maximum number of records per query** = *`200`*
- Open data license https://data.torontopolice.on.ca/pages/licence 
- Open data documentation (pdf saved in /raw_data folder)
    - The location of crime occurrences have been deliberately offset to the nearest road intersection node to protect the privacy of parties involved in the occurrence. All location data must be considered as an approximate location of the occurrence and users are advised not to interpret any of these locations as related to a specific address or individual.

In [2]:
import numpy as np
import pandas as pd
pd.set_option('display.max_columns', None)
import requests
import concurrent.futures
from datetime import datetime
from tqdm import tqdm
import shapely.geometry
import geopandas as gpd

In [3]:
# Projected Coordinate System (PCS): how you project onto a 2D plane
epsg_2d = 26717 
# Toronto Police Service Open Data includes geographic location information provided in
# the projected coordinate system NAD 1927 UTM 17N
# https://epsg.io/26717

# Geographic Coordinate System (GCS): how coordinates are represented in a 3D space
# World Geodetic System 1984 (WGS84)
# https://epsg.io/4326
epsg_3d = 4326 

# Datasets with API explorer
- Crime https://data.torontopolice.on.ca/datasets/TorontoPS::major-crime-indicators-1/about
- Traffic Collisions https://data.torontopolice.on.ca/datasets/TorontoPS::traffic-collisions-asr-t-tbl-001/about
- Shootings https://data.torontopolice.on.ca/datasets/TorontoPS::shootings-and-firearm-discharges/about
- Homicide https://data.torontopolice.on.ca/datasets/TorontoPS::homicide-asr-rc-tbl-002/about
- Persons in Crisis Calls for Service Attended https://data.torontopolice.on.ca/datasets/TorontoPS::persons-in-crisis-calls-for-service-attended/about
- Mental Health Act Apprehensions https://data.torontopolice.on.ca/datasets/TorontoPS::mental-health-act-apprehensions/about

In [10]:
# name of dataset to query from Toronto Police Service API
crimes = 'Major_Crime_Indicators'
traffic_collisions = 'Traffic_Collisions_ASR_T_TBL_001'
shootings = 'Shootings_and_Firearm_Discharges'
homicide = 'Homicide_ASR_RC_TBL_002'
pic_calls = 'PIC_Calls_for_Service_Attended'
mha_apprehensions = 'MHA_Apprehensions'
neighbourhoods = 'NeighbourhoodCrimeRates'

In [11]:
def get_objectIDs(data):
    url = f'https://services.arcgis.com/S9th0jAJ7bqgIRjw/arcgis/rest/services/{data}/FeatureServer/0/query?where=1%3D1&outFields=*&' + \
        'returnIdsOnly=true' + \
        '&outSR=4326&f=json'
    r = requests.get(url)
    json_data = r.json()
    objectIDs = sorted(json_data['objectIds'])
    print(f'number of records in {data}: {len(objectIDs):,.0f}')
    return objectIDs

In [12]:
# each record has a unique objectID which we can use as an identifier
crime_objectIDs = get_objectIDs(crimes)
collisions_objectIDs = get_objectIDs(traffic_collisions)
shootings_objectIDs = get_objectIDs(shootings)
homicide_objectIDs = get_objectIDs(homicide)
pic_calls_objectIDs = get_objectIDs(pic_calls)
mha_apprehensions_objectIDs = get_objectIDs(mha_apprehensions)
neighbourhoods_objectIDs = get_objectIDs(neighbourhoods)

number of records in Major_Crime_Indicators: 281,692
number of records in Traffic_Collisions_ASR_T_TBL_001: 526,256
number of records in Shootings_and_Firearm_Discharges: 5,328
number of records in Homicide_ASR_RC_TBL_002: 1,252
number of records in PIC_Calls_for_Service_Attended: 191,460
number of records in MHA_Apprehensions: 67,958
number of records in NeighbourhoodCrimeRates: 140


In [13]:
# function to request data from API given the object ID
def parallel_request(objectIDs, data, columns = '*', chunk_size = 200):
    output = None
    # splitting the objectIDs into chunks of 200
    objectIDs_chunks = [objectIDs[i:i+chunk_size] for i in range(0,len(objectIDs),chunk_size)]
    
    # function to get data for each objectID chunk
    def get_data(id_list, data, columns):
        hdr = {'User-Agent': "Mozilla/5.0"}
        bucket = ','.join(map(str, id_list))
        url = 'https://services.arcgis.com/S9th0jAJ7bqgIRjw/arcgis/rest/services/' + \
            f'{data}/' + \
            'FeatureServer/0/query?objectIds=' + bucket + \
            f'&outFields={columns}' + \
            '&outSR=4326&f=json&returnExceededLimitFeatures=true'
        r = requests.get(url, headers=hdr)
        try:
            r.raise_for_status()
            json_data = r.json()
            return pd.json_normalize(json_data['features'])    
        except requests.HTTPError as exception:
            print(f'error with chunk {id_list[0]}')
            print(exception)
            return None
    
    with tqdm(total=len(objectIDs_chunks)) as pbar:
        # We can use a with statement to ensure threads are cleaned up promptly
        with concurrent.futures.ThreadPoolExecutor() as executor:
            # Start the load operations and mark each future with its object_ID
            future_to_object_ID = {executor.submit(get_data, object_IDs, data, columns): chunk_n for chunk_n, object_IDs in enumerate(objectIDs_chunks)}
            for future in concurrent.futures.as_completed(future_to_object_ID):
                chunk_n = future_to_object_ID[future]
                # print(f'{(chunk_n + 1)/len(objectIDs_chunks)*100:0.0f}% completed')
                try:
                    output = pd.concat([output, future.result()], axis=0)
                except Exception as exc:
                    print('Chunk %r generated an exception: %s' % (chunk_n, exc))
                    return
                # else:
                #     print('%r page is %d bytes' % (chunk_n, len(future.result())))
                pbar.update(1)
    return output

In [10]:
# test
parallel_request(crime_objectIDs[:10**4], crimes)

100%|██████████| 50/50 [00:06<00:00,  7.56it/s]


,attributes.Index_,attributes.event_unique_id,attributes.Division,attributes.occurrencedate,attributes.reporteddate,attributes.location_type,attributes.premises_type,attributes.ucr_code,attributes.ucr_ext,attributes.offence,attributes.reportedyear,attributes.reportedmonth,attributes.reportedday,attributes.reporteddayofyear,attributes.reporteddayofweek,attributes.reportedhour,attributes.occurrenceyear,attributes.occurrencemonth,attributes.occurrenceday,attributes.occurrencedayofyear,attributes.occurrencedayofweek,attributes.occurrencehour,attributes.MCI,attributes.Hood_ID,attributes.Neighbourhood,attributes.Long,attributes.Lat,attributes.ObjectId,geometry.x,geometry.y
0,110,GO-20141625305,D23,1393736400000,1393736400000,"Single Home, House (Attach Garage, Cottage, Mo...",House,1430,100,Assault,2014,March,2,61,Sunday,8,2014,March,2,61,Sunday,8,Assault,1,West Humber-Clairville,-79.590332,43.734013,1,-79.590332,43.734013
1,188,GO-20141272968,D23,1387861200000,1388725200000,"Commercial Dwelling Unit (Hotel, Motel, B & B,...",Commercial,1610,200,Robbery - Mugging,2014,January,3,3,Friday,11,2013,December,24,358,Tuesday,22,Robbery,1,West Humber-Clairville,-79.600701,43.731834,2,-79.600701,43.731834
2,287,GO-20141284361,D23,1357362000000,1388898000000,"Commercial Dwelling Unit (Hotel, Motel, B & B,...",Commercial,1430,100,Assault,2014,January,5,5,Sunday,5,2013,January,5,5,Saturday,4,Assault,1,West Humber-Clairville,-79.600794,43.686423,3,-79.600794,43.686423
3,384,GO-20141292177,D23,1388466000000,1388984400000,Other Commercial / Corporate Places (For Profi...,Commercial,2120,200,B&E,2014,January,6,6,Monday,17,2013,December,31,365,Tuesday,21,Break and Enter,1,West Humber-Clairville,-79.603876,43.743642,4,-79.603876,43.743642
4,438,GO-20141297201,D23,1388725200000,1389070800000,Other Commercial / Corporate Places (For Profi...,Commercial,2120,200,B&E,2014,January,7,7,Tuesday,10,2014,January,3,3,Friday,10,Break and Enter,1,West Humber-Clairville,-79.586443,43.697108,5,-79.586443,43.697108
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
195,264700,GO-2019299231,D23,1550293200000,1550293200000,"Streets, Roads, Highways (Bicycle Path, Privat...",Outside,2135,210,Theft Of Motor Vehicle,2019,February,16,47,Saturday,16,2019,February,16,47,Saturday,1,Auto Theft,1,West Humber-Clairville,-79.575427,43.725357,6596,-79.575427,43.725357
196,264702,GO-2019299060,D23,1550293200000,1550293200000,"Parking Lots (Apt., Commercial Or Non-Commercial)",Outside,2135,210,Theft Of Motor Vehicle,2019,February,16,47,Saturday,16,2019,February,16,47,Saturday,15,Auto Theft,1,West Humber-Clairville,-79.605691,43.728929,6597,-79.605691,43.728929
197,264709,GO-2019295407,D23,1549602000000,1550293200000,"Parking Lots (Apt., Commercial Or Non-Commercial)",Outside,2135,210,Theft Of Motor Vehicle,2019,February,16,47,Saturday,0,2019,February,8,39,Friday,5,Auto Theft,1,West Humber-Clairville,-79.595527,43.686758,6598,-79.595527,43.686758
198,264719,GO-2019282482,D23,1548738000000,1550120400000,"Parking Lots (Apt., Commercial Or Non-Commercial)",Outside,2135,210,Theft Of Motor Vehicle,2019,February,14,45,Thursday,6,2019,January,29,29,Tuesday,17,Auto Theft,1,West Humber-Clairville,-79.587266,43.689517,6599,-79.587266,43.689517


In [11]:
# select the columns we need 
crime_columns = 'event_unique_id,occurrencedate,premises_type,occurrenceyear,occurrencemonth,occurrenceday,occurrencedayofyear,occurrencedayofweek,occurrencehour,MCI,hood_id,Neighbourhood,Long,Lat'
collisions_columns = 'EventUniqueId,OccurrenceDate,Month,Day_of_Week,Year,Hour,Atom,Neighbourhood,Fatalities,Injury_Collisions,FTR_Collisions,PD_Collisions,Longitude,Latitude'
pic_calls_columns = 'EventID,EventDate,EventYear,EventMonth,EventDayOfWeek,EventHour,EventType,OccurrenceCreated,ApprehensionMade,hood_id,NeighbourhoodName'
mha_apprehensions_columns = 'EventUniqueID,OccurrenceDate,OccurrenceYear,OccurrenceMonth,OccurrenceDayofWeek,OccurrenceHour,HoodID,NeighbourhoodName,PremisesType,Sex,AgeGroup'

In [12]:
crime_data = parallel_request(crime_objectIDs, crimes, crime_columns)

100%|██████████| 1409/1409 [02:17<00:00, 10.28it/s]


In [13]:
collisions_data = parallel_request(collisions_objectIDs, traffic_collisions, collisions_columns)

100%|██████████| 2632/2632 [07:44<00:00,  5.67it/s]


In [14]:
shootings_data = parallel_request(shootings_objectIDs, shootings)

100%|██████████| 27/27 [00:02<00:00, 10.18it/s]


In [15]:
homicide_data = parallel_request(homicide_objectIDs, homicide)

100%|██████████| 7/7 [00:00<00:00, 11.68it/s]


In [16]:
pic_calls_data = parallel_request(pic_calls_objectIDs, pic_calls, pic_calls_columns)

100%|██████████| 958/958 [01:17<00:00, 12.43it/s]


In [17]:
mha_apprehensions_data = parallel_request(mha_apprehensions_objectIDs, mha_apprehensions, mha_apprehensions_columns)

100%|██████████| 340/340 [00:21<00:00, 15.79it/s]


In [14]:
neighbourhoods_data = parallel_request(neighbourhoods_objectIDs, neighbourhoods)

100%|██████████| 1/1 [00:01<00:00,  1.84s/it]


# Clean column names

In [16]:
def clean_columns(df):    
    
    def convert_timestamp(x):
        if type(x) is pd.Timestamp:
            return x
        else:
            try:
                return datetime.fromtimestamp(x/1000)
            except:
                return np.nan

    df.columns = [x.split('.')[1] if '.' in x else x for x in df.columns]
    df.columns = [x.lower().replace('_', '') for x in df.columns]
    try:
        df['occurrencedate'] = df['occurrencedate'].apply(convert_timestamp)
    except: 
        pass
    
    try:
        df.rename(columns = {
            'agegroup': 'age_group',
            'apprehensionmade': 'apprehension_made',
            'atom': 'hood_id',
            'dayofweek': 'occurrence_dayofweek',
            'eventdate': 'occurrence_date',
            'eventdayofweek': 'occurrence_dayofweek',
            'eventhour': 'occurrence_hour',
            'eventid': 'occurrence_unique_id',
            'eventmonth': 'occurrence_month',
            'eventtype': 'occurrence_type',
            'eventuniqueid': 'occurrence_unique_id',
            'eventyear': 'occurrence_year',
            'ftrcollisions': 'ftr_collisions',
            'homicidetype': 'homicide_type',
            'hoodid': 'hood_id',
            'hour': 'occurrence_hour',
            'injurycollisions': 'injury_collisions',
            'lat': 'latitude',
            'long': 'longitude',
            'mci': 'MCI',
            'month': 'occurrence_month',
            'neighbourhoodname': 'neighbourhood',
            'objectid': 'object_id',
            'occurrencecreated': 'occurrence_created',
            'occurrencedate': 'occurrence_date',
            'occurrenceday': 'occurrence_day',
            'occurrencedayofweek': 'occurrence_dayofweek',
            'occurrencedayofyear': 'occurrence_dayofyear',
            'occurrencehour': 'occurrence_hour',
            'occurrencemonth': 'occurrence_month',
            'occurrencetype': 'occurrence_type',
            'occurrenceuniqueid': 'occurrence_unique_id',
            'occurrenceyear': 'occurrence_year',
            'pdcollisions': 'pd_collisions',
            'premisestype': 'premises_type',
            'timerange': 'time_range',
            'year': 'occurrence_year',
            }, inplace = True)
    except:
        pass        
    
    # some columns have "NULL" instead of NULL
    for x in df.columns:
        df[x] = df[x].replace('NULL', np.nan)
    
    return df

In [20]:
for df in [crime_data, collisions_data, shootings_data, homicide_data, pic_calls_data, mha_apprehensions_data, neighbourhoods_data]:
    df = clean_columns(df)

In [18]:
neighbourhoods_data['geometry'] = neighbourhoods_data.apply(lambda x: shapely.geometry.Polygon(x['rings'][0]), axis = 1)
neighbourhoods_data.drop(columns=['shapearea', 'shapelength', 'rings'], inplace = True)
neighbourhoods_data = gpd.GeoDataFrame(neighbourhoods_data, 
                                crs='epsg:' + str(epsg_3d), 
                                geometry='geometry')
print(type(neighbourhoods_data))

<class 'geopandas.geodataframe.GeoDataFrame'>


# Write to raw_data folder

In [22]:
crime_data.to_csv('../data/raw/Major_Crime_Indicators.csv', index = False)
collisions_data.to_csv('../data/raw/Traffic_Collisions.csv', index = False)
shootings_data.to_csv('../data/raw/Shootings.csv', index = False)
homicide_data.to_csv('../data/raw/Homicide.csv', index = False)
pic_calls_data.to_csv('../data/raw/Persons_in_Crisis_Calls_for_Service_Attended.csv', index = False)
mha_apprehensions_data.to_csv('../data/raw/Mental_Health_Act_Apprehensions.csv', index = False)

In [19]:
neighbourhoods_data.to_file('../data/raw/Neighbourhood_Boundary.geojson', driver='GeoJSON')

c:\Users\willi\miniconda3\envs\police\lib\site-packages\geopandas\io\file.py:362: FutureWarning: pandas.Int64Index is deprecated and will be removed from pandas in a future version. Use pandas.Index with the appropriate dtype instead.
  pd.Int64Index,
